# Exercício 1 - 8-Puzzle

Sua tarefa consiste de implementar três agentes guiados a objetivo para que resolvam o problema do 8-puzzle.

O agente BFS corresponde à busca em largura. Já o agente DFS corresponde à busca em profundidade. 

Por fim, o agente IDFS por aprofundamento iterativo.

Cabe ressaltar que, cada movimento apresenta custo 1. A função sucessora leva sempre a estados válidos.

## Imports
Para começar vamos importar algumas funções que vamos utilizar no código.

In [4]:
from collections import deque
import time

## Classe: Node
A classe Node vai representar cada estado que aconteceu durante a execução do algoritmo de busca. A partir do node final que temos as informações para formar o path da resposta. Temos dentro da classe o estado atual(state), o custo do percurso(cost), o movimento que fez para estar nesse estado(move) e o pai que originou o nó atual(parent).

In [2]:
class Node:
    def __init__(self, state, cost, move = "", parent= None) :
        self.parent = parent
        self.state = state
        self.move = move
        self.cost = cost

## Classe: Problem
A classe Problem vai representa a definição do nosso problema, contendo principalmente o teste de objetivo, as possíveis ações a se realizar e a função que implementa a realização da ação. Temos como atributos o 'initial_state', que representa o estado inicial que deve começar o problema, o 'goal', representando como deve ficar no final, e 'size', que é o tamanho da matriz quadrada que teremos.

In [3]:
class Problem:
    def __init__(self, size, initial_state, goal):
        self.initial_state = initial_state
        self.goal = goal
        self.size = size
    
    
    # Diz se o estado atual chegou ao objetivo.
    def objective_test(self, state):
        return state == self.goal

    # Pega a coordenada x a partir de um index do vetor
    def get_x(self, index):
        return index - self.get_y(index) * self.size

    # Pega a coordenada y a partir de um index do vetor
    def get_y(self, index):
        return index // self.size

    
    # Pega as coordenadas x e y a partir de um index do vetor
    def get_coords(self, index):
        return self.get_x(index), self.get_y(index)

    
    # Pega o index do vetor a partir das coordenadas x e y
    def get_index(self, x, y):
        return y * self.size + x

    
    # Pega as possíveis ações que o problema pode ter
    def actions(self, state):
        moves = ["Up", "Right", "Left", "Down"]
        index = state.index(0)
        x, y = self.get_coords(index)

        if x == 0:
            moves.remove("Left")
        elif x == self.size - 1:
            moves.remove("Right")

        if y == 0:
            moves.remove("Up")
        elif y == self.size - 1:
            moves.remove("Down")

        return moves

    
    # Faz o movimento 'action' no estado 'estate' e retorna o novo estado
    def move(self, state, action):
        result = state.copy()
        index = state.index(0)
        x, y = self.get_coords(index)

        if action == "Up":
            y -= 1
        elif action == "Down":
            y += 1
        elif action == "Left":
            x -= 1
        elif action == "Right":
            x += 1

        new_index = self.get_index(x, y)

        result[index], result[new_index] = result[new_index], result[index]

        return result

    
    # Pega o path que fez chegar no 'node'
    def get_path(self, node):
        moves = []

        while node.parent:
            moves.append(f"'{node.move}'")
            node = node.parent

        moves.reverse()
        return ", ".join(moves)

## Busca em Largura


In [4]:
def bfs(problem:Problem):
    maxdepth = 0
    maxfrontier = 0
    # Criando o primeiro nó
    node = Node(problem.initial_state, 0)
    
    if problem.objective_test(node.state):
        return {"node": node, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": 0, "scanned": 0}
    
    # Criando as listas que vão ser usadas
    edge = deque()
    edge.append(node)
    explored = set()
    
    # Enquanto edge não for vazia
    while edge:
        # Pega o primeiro valor que entrou na fila e adiciona nos explorados
        node = edge.popleft()
        explored.add(str(node.state))
        
        # Pega as possíveis ações que podemos fazer a partir do estado atual
        for action in problem.actions(node.state):
            
            # Realiza a ação e pega o novo estado
            newstate = problem.move(node.state, action)
            
            # Cria o filho do nó atual com a ação selecionada
            son = Node(newstate, node.cost + 1, action, node)
            
            # Checa se o filho já foi explorado
            if str(son.state) not in explored:
                # Gravando a variável de maior profundidade
                if son.cost > maxdepth:
                    maxdepth = son.cost

                # Se o filho for a resposta, retorna a solução
                if problem.objective_test(son.state):
                    return {"node": son, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}
                
                # Adiciona o filho a fila e na lista de explorados
                edge.append(son)
                explored.add(str(son.state))

        # Gravando a variável de maior fronteira
        if len(edge) > maxfrontier: 
            maxfrontier = len(edge)
        
        
    return {"node": None, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}


## Busca em profundidade

In [5]:
def dfs(problem:Problem):
    maxdepth = 0
    maxfrontier = 0
    # Criando o primeiro nó
    node = Node(problem.initial_state, 0)

    if problem.objective_test(node.state):
        return {"node": node, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": 0, "scanned": 0}

    # Criando as listas que vão ser usadas
    edge = deque()
    edge.append(node)
    explored = set()
    
    # Enquanto edge não for vazia
    while edge:
        # Pega o último valor que entrou na fila e adiciona nos explorados
        node = edge.pop()
        explored.add(str(node.state))

        # Pega as possíveis ações que podemos fazer a partir do estado atual
        for action in problem.actions(node.state):
            
            # Realiza a ação e pega o novo estado
            newstate = problem.move(node.state, action)

            # Cria o filho do nó atual com a ação selecionada
            son = Node(newstate, node.cost + 1, action, node)
            
            # Checa se o filho já foi explorado
            if str(son.state) not in explored:
                # Gravando a variável de maior profundidade
                if son.cost > maxdepth:
                    maxdepth = son.cost

                # Se o filho for a resposta, retorna a solução
                if problem.objective_test(son.state):
                    return {"node": son, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}

                # Adiciona o filho a fila e na lista de explorados
                edge.append(son)
                explored.add(str(son.state))

        # Gravando a variável de maior fronteira
        if len(edge) > maxfrontier: 
            maxfrontier = len(edge)
        
        
    return {"node": None, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}

## Busca por aprofundamento iterativo

In [6]:
def idfs(problem:Problem, minimum, maximum, step):
    
    # Criação da iteração entre minimo e maximo com o step
    for limit in range(minimum, maximum, step):

        maxdepth = 0
        maxfrontier = 0
        # Criando o primeiro nó
        node = Node(problem.initial_state, 0)

        if problem.objective_test(node.state):
            return {"node": node, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": 0, "scanned": 0}

        # Criando as listas que vão ser usadas
        edge = deque()
        edge.append(node)
        explored = set()
        
        # Enquanto edge não for vazia
        while edge:
            # Pega o último valor que entrou na fila e adiciona nos explorados
            node = edge.pop()
            explored.add(str(node.state))
            
            # Checar se o custo passou do limite e caso passe pegar o próximo limite
            if node.cost > limit:
                break
        
            # Pega as possíveis ações que podemos fazer a partir do estado atual
            for action in problem.actions(node.state):

                # Realiza a ação e pega o novo estado
                newstate = problem.move(node.state, action)

                # Cria o filho do nó atual com a ação selecionada
                son = Node(newstate, node.cost + 1, action, node)
                
                # Checa se o filho já foi explorado
                if str(son.state) not in explored:
                    # Gravando a variável de maior profundidade
                    if son.cost > maxdepth:
                        maxdepth = son.cost
                        
                    # Se o filho for a resposta, retorna a solução
                    if problem.objective_test(son.state):
                        return {"node": son, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}

                    # Adiciona o filho a fila e na lista de explorados
                    edge.append(son)
                    explored.add(str(son.state))



            # Gravando a variável de maior fronteira
            if len(edge) > maxfrontier: 
                maxfrontier = len(edge)
        
        
    return {"node": None, "maxdepth": maxdepth, "maxfrontier": maxfrontier, "finalfrontier": len(edge), "scanned": len(explored)}

## Rodando os testes
Utilizando as classes para criar os testes pedidos para o 8-puzzle, iniciando em [0, 8, 7, 6, 5, 4, 3, 2, 1] com o objetivo de [0, 1, 2, 3, 4, 5, 6, 7, 8].

In [7]:
problem = Problem(3, [0, 8, 7, 6, 5, 4, 3, 2, 1], [0, 1, 2, 3, 4, 5, 6, 7, 8])

start_time_bfs = time.time()
result_bfs = bfs(problem)
final_time_bfs = time.time()


start_time_dfs = time.time()
result_dfs = dfs(problem)
final_time_dfs = time.time()

start_time_idfs = time.time()
result_idfs = idfs(problem, 1000, 15000, 1000)
final_time_idfs = time.time()


## Mostrando resultados
Observações:
- O path do dfs e idfs foram escondidos por serem muito grandes e deixarem o notebook ilegível.
- A ram usada será calculada apenas nos arquivos separados para chegarmos a um resultado mais justo.

In [8]:
node = result_bfs['node']
print(f"BFS:")
print(f"\tpath_to_goal: [{problem.get_path(node)}]")
print(f"\tcost_of_path: {node.cost}")
print(f"\tnodes_expanded: {result_bfs['scanned']}")
print(f"\tfringe_size: {result_bfs['finalfrontier']}")
print(f"\tmax_fringe_size: {result_bfs['maxfrontier']}")
print(f"\tsearch_depth: {node.cost}")
print(f"\tmax_search_depth: {result_bfs['maxdepth']}")
print(f"\trunning_time: {final_time_bfs - start_time_bfs} s")
print(f"\tmax_ram_usage: Apresentada no arquivo para melhor precisão")

print()
print()

node = result_dfs['node']
print(f"DFS:")
print(f"\tpath_to_goal: Apresentado no arquivo devido ao tamanho")
# Descomentar a linha abaixo caso queira ver o path no notebook
# print(f"\tpath_to_goal: [{problem.get_path(node)}]")
print(f"\tcost_of_path: {node.cost}")
print(f"\tnodes_expanded: {result_dfs['scanned']}")
print(f"\tfringe_size: {result_dfs['finalfrontier']}")
print(f"\tmax_fringe_size: {result_dfs['maxfrontier']}")
print(f"\tsearch_depth: {node.cost}")
print(f"\tmax_search_depth: {result_dfs['maxdepth']}")
print(f"\trunning_time: {final_time_dfs - start_time_dfs} s")
print(f"\tmax_ram_usage: Apresentada no arquivo para melhor precisão")

print()
print()

node = result_idfs['node']
print(f"IDFS:")
print(f"\tmin_limit: 1000")
print(f"\tmax_limit: 15000")
print(f"\tstep: 1000")
print(f"\tpath_to_goal: Apresentado no arquivo devido ao tamanho")
# Descomentar a linha abaixo caso queira ver o path no notebook
# print(f"\tpath_to_goal: [{problem.get_path(node)}]")
print(f"\tcost_of_path: {node.cost}")
print(f"\tnodes_expanded: {result_idfs['scanned']}")
print(f"\tfringe_size: {result_idfs['finalfrontier']}")
print(f"\tmax_fringe_size: {result_idfs['maxfrontier']}")
print(f"\tsearch_depth: {node.cost}")
print(f"\tmax_search_depth: {result_idfs['maxdepth']}")
print(f"\trunning_time: {final_time_idfs - start_time_idfs} s")
print(f"\tmax_ram_usage: Apresentada no arquivo para melhor precisão")

BFS:
	path_to_goal: ['Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left']
	cost_of_path: 30
	nodes_expanded: 181363
	fringe_size: 492
	max_fringe_size: 24051
	search_depth: 30
	max_search_depth: 30
	running_time: 3.2763822078704834 s
	max_ram_usage: Apresentada no arquivo para melhor precisão


DFS:
	path_to_goal: Apresentado no arquivo devido ao tamanho
	cost_of_path: 9964
	nodes_expanded: 19948
	fringe_size: 8834
	max_fringe_size: 8835
	search_depth: 9964
	max_search_depth: 9964
	running_time: 0.20756030082702637 s
	max_ram_usage: Apresentada no arquivo para melhor precisão


IDFS:
	min_limit: 1000
	max_limit: 15000
	step: 1000
	path_to_goal: Apresentado no arquivo devido ao tamanho
	cost_of_path: 9964
	nodes_expanded: 19948
	fringe_size: 8834
	max_fringe_size: 8835
	search_depth: 9964
	max_search_depth: 9964